<a href="https://colab.research.google.com/github/ajaysuseel/MiniProject_AD/blob/main/hazard_detection_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
# Import BLIP processor and model (adjust these imports based on your BLIP implementation)
from transformers import BlipProcessor, BlipForConditionalGeneration

# Load the finetuned BLIP model and processor
blip_processor = BlipProcessor.from_pretrained("models/finetunedblip1")
blip_model = BlipForConditionalGeneration.from_pretrained("models/finetunedblip1")

# Load a lightweight LLM (e.g., distilgpt2)
llm_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
llm_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

def generate_caption(image_path):
    """
    Generate a caption for the input image using the finetuned BLIP model.
    """
    image = Image.open(image_path).convert("RGB")
    inputs = blip_processor(image, return_tensors="pt")
    outputs = blip_model.generate(**inputs)
    caption = blip_processor.decode(outputs[0], skip_special_tokens=True)
    return caption

def detect_hazard(caption):
    """
    Use the lightweight LLM to determine if the caption indicates a hazard.
    The prompt instructs the LLM to answer with 'Hazard' or 'No Hazard' along with a brief explanation.
    """
    prompt = (
        f"Determine if the following caption describes a hazard scenario in an autonomous driving context.\n\n"
        f"Caption: '{caption}'\n\n"
        f"Answer with 'Hazard' or 'No Hazard' and provide a brief explanation."
    )

    # Tokenize the prompt
    input_ids = llm_tokenizer(prompt, return_tensors="pt").input_ids

    # Generate the LLM response
    output_ids = llm_model.generate(input_ids, max_length=100, do_sample=True, temperature=0.7)
    response = llm_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Parse the response (simple parsing; adjust as needed)
    if "Hazard" in response:
        hazard = True
    elif "No Hazard" in response:
        hazard = False
    else:
        hazard = None  # Unclear outcome

    return hazard, response

def main(image_path):
    # Generate caption from the image
    caption = generate_caption(image_path)
    print("Generated Caption:", caption)

    # Use the caption to detect hazard
    hazard, explanation = detect_hazard(caption)
    print("Hazard Detected:", hazard)
    print("LLM Explanation:", explanation)

if __name__ == "__main__":
    # Replace with your image file path
    image_file = "path_to_your_image.jpg"
    main(image_file)
